In [53]:
from pathlib import Path
from utils import *
import pandas as pd
import pyodbc
import string
import re
import spacy
from pprint import pprint
from tqdm import tqdm
import fuzzywuzzy
from fuzzywuzzy import fuzz, process
from unidecode import unidecode


DATA_FOLDER = Path("C:/Users/Jean-BaptistePERNEY/Documents/ECHO/DATA/2022_audio_database/big_db_test/processed_files/")

### Identify and save personnal information from transcripts

In [113]:
def is_valid_name(name):
    """Check if the name is valid for matching."""
    cleaned_name = name.strip()
        # Exclude names that are too short
    if len(cleaned_name) <= 2:
        return False
    
    # Exclude specific non-name words or symbols
    non_name_keywords = ["sos", "oui", "allô", "médecin", "médecins", "bonjour", "covid"]  # Add more keywords as needed
    if cleaned_name.lower() in non_name_keywords:
        return False

    return True

def find_names_with_spacy(transcript):
    nlp = spacy.load('fr_dep_news_trf')
    doc = nlp(transcript)
    names = list(set([w.text for w in doc if w.pos_ == 'PROPN' and is_valid_name(w.text)]))
    
    return names

def month_name_to_number(month_name):
    month_names = ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 
                   'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre']
    try:
        return month_names.index(month_name.lower()) + 1
    except ValueError:
        return None
    

def extract_full_date(transcript):
    min_year = 1900

    # Pattern for numerical full date
    full_date_pattern = r'\b(0?[1-9]|[12][0-9]|3[01])[-/](0?[1-9]|1[0-2])[-/](\d{2}|\d{4})\b'
    
    # Pattern for full date with month name
    month_names = '|'.join(['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 
                            'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre'])
    date_with_month_pattern = r'\b(0?[1-9]|[12][0-9]|3[01])\s+(' + month_names + r')\s+(\d{4})\b'

    # Try to extract full date in numerical format
    full_dates = re.findall(full_date_pattern, transcript)
    formatted_dates = []
    if full_dates:
        for date in full_dates:
            day, month, year = date
            if len(year) == 2:  # Convert two-digit year to four digits
                year = "19" + year if year > "24" else "20" + year
            # Handle transcript erros (ex: 1568 instead of 1968)

            if int(year) < min_year:
                year = int("19" + year[-2:])

            formatted_dates.append(f"{day.zfill(2)}/{month.zfill(2)}/{str(year)}")
    
        return formatted_dates

    # Try to extract full date with month name
    dates_with_month = re.findall(date_with_month_pattern, transcript)
    if dates_with_month:
        for date in dates_with_month:
            day, month_name, year = date
            month_number = month_name_to_number(month_name)
            if month_number:
                formatted_dates.append(f"{day.zfill(2)}/{month_number:02d}/{year}")
        
        return formatted_dates

    # Extract just the year
    year_pattern = r'\b(19\d{2}|20\d{2})\b'
    years = re.findall(year_pattern, transcript)
    if years:
        year = int(years[0])
        if year < min_year:
            year = int("19" + year[-2:])
        return f"01/01/{str(year)}"  # Defaulting to '01/01/YYYY' for year-only cases

    return None

def extract_phone_numbers(transcript):
    phone_regex = r'(06|07)(\d{8})'

    transcript = transcript.replace(" ", "")
    transcript = transcript.translate(str.maketrans('', '', string.punctuation))
    match = re.search(phone_regex, transcript)
    match = match.group() if match else match
    return match

def extract_postcodes(transcript):
    # Existing pattern for postcodes like '75015' or '75-015'
    postcode_pattern = r'\b(75|77|78|91|92|93|94|95)(?:[-\s]?)(\d{3})\b'
    raw_postcodes = re.findall(postcode_pattern, transcript)

    # Additional pattern for postcodes written as '19ème'
    arrondissement_pattern = r'\b(\d{1,2})[eè]me\b'
    arrondissements = re.findall(arrondissement_pattern, transcript)

    # Formatting arrondissements to Paris postcodes (e.g., '75019' for '19ème')
    formatted_arrondissements = ['75' + arr.zfill(3) for arr in arrondissements]

    # Combining and returning all found postcodes
    return [''.join(match) for match in raw_postcodes] + formatted_arrondissements

def extract_personal_info(data_folder):
    personal_info = {}

    address_pattern = r'\b(\d+\s+(?:rue|rues|boulevard|avenue|place)\s+[a-zA-Zéèàêûôâîç\s-]+)\b'
    address_pattern = r'\b(\d+\s+(?:rue|rues|boulevard|avenue|place)\s+[a-zA-Zéèàêûôâîç\s-]+(?:\'[a-zA-Zéèàêûôâîç\s-]+)?)\b'
    # address_pattern = r'\b\d+\s*(rue|rues|avenue|boulevard|place)\s*[^\d,]+\b'
    number_sequence_pattern = r'\b\d+\b'

    for file_path in tqdm(data_folder.rglob('*.txt')):
        with open(file_path, 'r', encoding='utf-8') as file:
            transcript = file.read()

            personal_info[str(Path(file_path.name))] = {
                'date_of_birth': {
                    'full_date': extract_full_date(transcript)
                },
                'addresses': re.findall(address_pattern, transcript),
                'postcodes': extract_postcodes(transcript),
                'phone_numbers': extract_phone_numbers(transcript),
                # 'names' : list(set(find_names_with_spacy(transcript))),
                # 'other_findings': re.findall(number_sequence_pattern, transcript),
            }
        
    return personal_info

personnal_data = extract_personal_info(DATA_FOLDER)

17162it [00:57, 299.42it/s]


In [114]:
def calculate_personal_data_stats(personal_data):
    stats = {
        'total_files': 0,
        'non_empty_files': 0,
        'date_of_birth': 0,
        'addresses': 0,
        'postcodes': 0,
        'phone_numbers': 0
    }

    for _, info in personal_data.items():
        stats['total_files'] += 1
        non_empty = False

        for key, value in info.items():
            # Special handling for 'date_of_birth' as it contains a nested dictionary
            if key == 'date_of_birth':
                if value.get('full_date'):  # Check if 'full_date' is not empty
                    non_empty = True
                    stats[key] += 1
            elif value:  # Checks if the item is non-empty for other keys
                non_empty = True
                stats[key] += len(value) if isinstance(value, list) else 1

        if non_empty:
            stats['non_empty_files'] += 1

    return stats

# Usage
stats = calculate_personal_data_stats(personnal_data)
print(stats)

{'total_files': 17162, 'non_empty_files': 11415, 'date_of_birth': 6601, 'addresses': 8781, 'postcodes': 8489, 'phone_numbers': 2283}


### Load telecom log database

In [6]:
def load_mdb_data(file_path, table_name="InboundVoiceCalls"):
    try:
        # Connection string for MDB file
        conn_str = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={file_path};'
        
        # Establishing connection to the MDB file
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        
        # Reading each table into a pandas DataFrame
        df = pd.read_sql(f'SELECT * FROM [{table_name}]', conn)
        
        cursor.close()
        conn.close()

        return df
        
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

file_path = "/Users/Jean-BaptistePERNEY/Documents/ECHO/DATA/telecom_log_2022/01022022.mdb"
telecom_log_df = load_mdb_data(file_path, table_name="InboundVoiceCalls")

telecom_log_df = telecom_log_df[telecom_log_df['CallAgentCommunicationDuration'] != 0]
telecom_log_df = telecom_log_df[telecom_log_df['CallServiceID'] != 'SOS']

columns_to_keep = ["CallTime", "CallCLID", "AgentID", "CallAgentCommunicationDuration"]
telecom_log_df = telecom_log_df[columns_to_keep]

telecom_log_df.head()

C:\Users\Jean-BaptistePERNEY\AppData\Local\Temp\ipykernel_4596\3679646185.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM [{table_name}]', conn)


,CallTime,CallCLID,AgentID,CallAgentCommunicationDuration
38,2021-12-11 08:24:43,00667513808,martynowski2438,118
49,2021-12-11 09:00:07,00689634034,martynowski2438,99
72,2021-12-11 10:15:33,00614477661,martynowski2438,165
100,2021-12-11 10:53:13,00783847741,martynowski2438,224
103,2021-12-11 10:57:41,00619819855,martynowski2438,115


### Data matching

In [115]:
import pandas as pd
from datetime import datetime
import re


def find_matching_row(csv_path, data_dict):

    # Read the CSV just once
    df = pd.read_csv(csv_path, sep=',',  encoding='utf-8' )

    # Explicitly convert the 'DateAppel' column to datetime objects
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)
    df['FullAdresse'].apply(lambda x: unidecode(str(x).lower()).replace("-", " "))

    # Prepare results dictionary
    results = {}

    for file_path, info in data_dict.items():
        # Parse the timestamp from the file_path
        timestamp_parts = file_path.split('_')
        date_str = f"{timestamp_parts[0]}-{timestamp_parts[1]}-{timestamp_parts[2]}"
        search_date = pd.to_datetime(date_str).date()

        # Filter the dataframe by the date
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        date_filtered_df = df[df['Date'].dt.date == search_date]

        date_filtered_df = date_filtered_df.copy()
        date_filtered_df['DateNaiss'] = pd.to_datetime(date_filtered_df['DateNaiss'], format='%Y-%m-%d', errors='coerce')

        # Attempt to match phone number
        if info['phone_numbers']:   
            transcript_phone_number = info['phone_numbers']
            medical_db_phone_number = date_filtered_df['TelPatient'].str.replace('.', '', regex=False).str.contains(transcript_phone_number, na=False)
            match = date_filtered_df[medical_db_phone_number]
            if not match.empty:
                results[file_path] = (match, "phone_numbers")
                continue  # Found a match with phone number, skip to next entry

        # Attempt to match addresses
        if info['addresses']:
            for address in info['addresses']:
                address = unidecode(str(address).lower()).replace("-", " ")
                list_adresse = date_filtered_df['FullAdresse'].tolist()
                best_match = process.extract(address, list_adresse, scorer=fuzz.token_sort_ratio)[0]
                if best_match[1] > 90:
                    match = date_filtered_df[date_filtered_df['FullAdresse'].str.contains(best_match[0], na=False)]
                    if not match.empty:
                        results[file_path] = (match, "addresses")
                        break  # Found a match with address, skip to next entry

                # Attempt to match date of birth
        if info['date_of_birth']['full_date']:
            for dob in info['date_of_birth']['full_date']:
                dob = pd.to_datetime(dob, dayfirst=True, errors='coerce')
                if dob is not pd.NaT:
                    match = date_filtered_df[date_filtered_df['DateNaiss'] == dob]
                    if not match.empty:
                        results[file_path] = (match, "date_of_birth")
                        continue  # Found a match with date of birth, skip to next entry

        # Attempt to match postcodes
        if info['postcodes']:
            for postcode in info['postcodes']:
                match = date_filtered_df[date_filtered_df['CodePostal'] == str(postcode)]
                if not match.empty:
                    results[file_path] = (match, "postcodes")
                    break  # Found a match with postcode, skip to next entry

        # # Attempt to match names
        # if 'names' in info and info['names']:
        #     for name in info['names']:
        #         match = date_filtered_df[(date_filtered_df['Nom'].str.contains(name, na=False)) | 
        #                                  (date_filtered_df['Prenom'].str.contains(name, na=False))]
        #         if not match.empty:
        #             results[file_path] = (match, "names")
        #             break  # Found a match with name, skip to next entry

    return results

# Example usage:
medical_db_path = "/Users/Jean-BaptistePERNEY/Documents/ECHO/DATA/medical_database_2022/BaseMed2022LR_cleaned.csv"

matches = find_matching_row(medical_db_path, personnal_data)
print(len(matches))
# for file_path, matched_rows in matches.items():
#     print(f"File: {file_path}")
#     print(matched_rows[0]["IdAppel"], matched_rows[1])

C:\Users\Jean-BaptistePERNEY\AppData\Local\Temp\ipykernel_4596\1919965185.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, sep=',',  encoding='latin-1' )


2910


In [110]:
count = 0 
for key, item in matches.items():
    if item[0]['Devenir'].values[0].astype('int') != 0:
        count += 1
print(count)

107


C:\Users\Jean-BaptistePERNEY\AppData\Local\Temp\ipykernel_4596\4119757077.py:3: RuntimeWarning: invalid value encountered in cast
  if item[0]['Devenir'].values[0].astype('int') != 0:


In [81]:
print(f"On réussi a matcher {len(matches)} fichiers sur {stats['total_files']} au total, soit : {len(matches) / stats['total_files'] * 100:.02f} % de la base")
print(f"On réussi a matcher {len(matches)} fichiers sur {stats['non_empty_files']} fichier ayant des infos personnelles récupèrable, soit : {len(matches) / stats['non_empty_files'] * 100:.02f} % de la base")
print(f"On réussi a matcher {len(matches)} fichiers sur {stats['addresses']} fichier avec adresse récupèrable, soit : {len(matches) / stats['addresses'] * 100:.02f} % de la base")

On réussi a matcher 4128 fichiers sur 17162 au total, soit : 24.05 % de la base
On réussi a matcher 4128 fichiers sur 11351 fichier ayant des infos personnelles récupèrable, soit : 36.37 % de la base
On réussi a matcher 4128 fichiers sur 8425 fichier avec adresse récupèrable, soit : 49.00 % de la base


In [41]:
matches['2022_2_10_12_27_17_ch25_transcription.txt']

(       Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  IdAppel          Nom Prenom  \
 19295         19295         19295       19295  6417465  BOU OUACHMA  Inaya   
 
           TelAppelant      TelPatient IdProvenance NumRue  ...       Date  \
 19295  06.69.09.33.29  06.69.09.33.29           71    110  ... 2022-02-10   
 
         Annee Mois  Jour Heure   Min AppelTiers  Age Devenir  \
 19295  2022.0  2.0  10.0  12.0  28.0        0.0  0.0     0.0   
 
                                          FullAdresse  
 19295  110 avenue du gÃÂ£ÃÂ©nÃÂ£ÃÂ©ral de gaulle  
 
 [1 rows x 32 columns],
 'date_of_birth')

In [42]:
not_matched = []

for file_path in tqdm(DATA_FOLDER.rglob('*.txt')):
    if file_path.name not in matches.keys():
        not_matched.append(file_path.name)

17162it [00:01, 14962.89it/s]


In [48]:
i = 5
print(personnal_data[not_matched[i]])
print(not_matched[i])

{'date_of_birth': {'full_date': None}, 'addresses': ['1 rue Firmin-Génier', '1 rue Firmin-Génier'], 'postcodes': ['75018', '75018', '75018'], 'phone_numbers': None}
2022_2_10_11_25_12_ch26_transcription.txt


In [52]:
Str_A = '1 rue Firmin-Génier'
Str_B = '1 rue firmin gemier' 
ratio = fuzz.ratio(unidecode(Str_A.lower()).replace("-", " "), unidecode(Str_B.lower()).replace("-", " "))
print('Similarity score: {}'.format(ratio))

Similarity score: 95


In [122]:
import locale 

csv_path = "C:/Users/Jean-BaptistePERNEY/Documents/ECHO/DATA/medical_database_2022/BaseMed2022LR.csv"
df_patients = pd.read_csv(csv_path, sep=',',  encoding='latin-1' )

df_patients['TelAppelant'] = df_patients['TelAppelant'].str.replace('.', '')
df_patients['TelPatient'] = df_patients['TelPatient'].str.replace('.', '')
df_patients['FullAdresse'] = df_patients['NumRue'].astype(str) + ' ' + df_patients['Adresse']
df_patients['FullAdresse'] = df_patients['FullAdresse'].apply(lambda x: unidecode(str(x).lower()).replace("-", " "))

# df_patients = df_patients.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'])
df_patients.head()
df_patients.to_csv("C:/Users/Jean-BaptistePERNEY/Documents/ECHO/DATA/medical_database_2022/BaseMed2022LR_cleaned.csv", encoding='utf-8', index=False)

C:\Users\Jean-BaptistePERNEY\AppData\Local\Temp\ipykernel_4596\2119069524.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_patients = pd.read_csv(csv_path, sep=',',  encoding='latin-1' )


In [123]:
csv_path = "C:/Users/Jean-BaptistePERNEY/Documents/ECHO/DATA/medical_database_2022/BaseMed2022LR_cleaned.csv"
df_patients = pd.read_csv(csv_path, sep=',', encoding='utf-8')
df_patients.head()

C:\Users\Jean-BaptistePERNEY\AppData\Local\Temp\ipykernel_4596\3843054379.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_patients = pd.read_csv(csv_path, sep=',', encoding='utf-8')


,IdAppel,Nom,Prenom,TelAppelant,TelPatient,IdProvenance,NumRue,Adresse,CodePostal,Ville,...,Date,Annee,Mois,Jour,Heure,Min,AppelTiers,Age,Devenir,FullAdresse
0,6349973,MIYEKE,Erwan,0617746257,0617746257,71,18,Rue broussais,75014.0,Paris 14E,...,01/01/2022,2022.0,1.0,1.0,0.0,57.0,0.0,0.0,0.0,18 rue broussais
1,6349982,MERCIER - MIHELIC,Alix elettra,0617170454,0617170454,71,126,Rue de rennes,75006.0,Paris 6e,...,01/01/2022,2022.0,1.0,1.0,1.0,52.0,0.0,10.0,0.0,126 rue de rennes
2,6349984,MULOT,Mariam,0634706225,0634706225,71,5,Rue du jura,75013.0,Paris 13E,...,01/01/2022,2022.0,1.0,1.0,2.0,0.0,0.0,47.0,1.0,5 rue du jura
3,6349986,MOINEAU,Denis,0681234991,0681234991,71,29,Rue du docteur finlay,75015.0,Paris 15E,...,01/01/2022,2022.0,1.0,1.0,2.0,16.0,0.0,58.0,1.0,29 rue du docteur finlay
4,6349988,FAUCHER,Sandrine,0647655164,0647655164,71,21,Rue gracieuse,75005.0,Paris 5e,...,01/01/2022,2022.0,1.0,1.0,2.0,35.0,0.0,37.0,1.0,21 rue gracieuse
